# EPAI3 Session 11 - Iterables and Iterators (Part 1)
While Sequences allow only refer to ordered group of items. These include Lists, Tuples and Strings. Some of the operations that sequences support include:

1. Concatenation
2. Repetitions
3. "in" and "not in" operators
4. Element wise indexing/Slicing.

It is important to note that all Sequences are "iterable" but it is not necessary that all iterables are sequences. E.g: Unordered Sets vs Lists. Further, **range ** objects are additionally restrictive as they don't permit concatenation/reptition.

In [ ]:
!git clone https://github.com/rajy4683/session11-EPAI3-rajy4683.git

In [1]:
!pwd

/mnt/c/Users/rajy/OneDrive - Nokia/EVA4P2/EPAI3/s11


In [1]:
%load_ext autoreload
%autoreload 2
import sys
from functools import lru_cache
import math
import numbers
import decimal
import sys

from polygonlib import ConvexPolygon
from polygonsequence import PolygonSequences


In [2]:
str(ConvexPolygon), str(PolygonSequences)

("<class 'polygonlib.ConvexPolygon'>",
 "<class 'polygonsequence.PolygonSequences'>")

In [3]:
PolygonSequences

polygonsequence.PolygonSequences

In [4]:
dir(PolygonSequences)

['PolygonSqIterator',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_polygonator',
 'check_number_type',
 'get_max_efficiency_poly']

## Polygons Iterators
Implement an iterable and an iterator to 

Refactor the `Polygons` (sequence) type, into an **iterable** and implement both an iterable, and an iterator.


In [5]:
from polygonlib import ConvexPolygon
from functools import lru_cache
import math
import numbers
import decimal

class PolygonSequences():
    '''
    Polygon iterable class that generates all polygons for a given circumradius and upto the max edges specified by n_sides.
    E.g: If input radius = 4 and n_sides = 10, polygons starting from sides = 3,4,...10 of size 4 will be generated
    Returns iterable of ConvexPolygon class.
    '''
    def check_number_type(self, x):
        if isinstance(x, bool):
            return False
        if isinstance(x, complex):
            return False
        return isinstance(x, numbers.Number)
    
    def __init__(self, n_sides, circumradius):
        """
        n_sides and circumradius are expected to be numbers
        """
        if (self.check_number_type(n_sides) == False or
            self.check_number_type(circumradius) == False):
            raise TypeError
                
        if n_sides < 3:
            raise ValueError(f'Minimum number of sides should be 3. Input n_sides={n_sides} ')
            
        self.max_poly_edges = n_sides
        self.circumradius = circumradius
        self._offset_to_polysides =  { idx:PolygonSequences._polygonator(i, self.circumradius) for idx,i in enumerate(range(3, self.max_poly_edges+1, 1)) }
        self.max_efficient_poly = self.get_max_efficiency_poly()
    
    def get_max_efficiency_poly(self):
        '''
        Returns the max efficient polygon. We can make a simple hack to use the one with largest number of edges.
        '''
        max_pol_ratio = self._offset_to_polysides[0].get_area/self._offset_to_polysides[0].get_perimeter
        max_pol_offset = 0
        for k,v in self._offset_to_polysides.items():
            curr_ratio = v.get_area/v.get_perimeter
            if curr_ratio > max_pol_ratio:
                max_pol_offset = k
                max_pol_ratio = curr_ratio
        return self._offset_to_polysides[max_pol_offset]
    def __repr__(self):
        '''
        Repr for Polygon sequence
        '''
        return f"Instance of PolygonSequences class. Max Edges:{self.max_poly_edges} Radius:{self.circumradius} Len: {len(self._offset_to_polysides)}"
    def __len__(self):
        '''
        Total length of the sequence. This will be equal to the max value provided during sequence creation.
        '''
        return len(self._offset_to_polysides)    
    def __getitem__(self, s):
        '''
        Allows for proper iteration of the sequence.
        '''
        if isinstance(s, int):
            if s < 0:
                s = self.__len__() + s
            if s < 0 or s >=self.__len__():
                raise IndexError(f"Invalid Index.")
            else:
#                 return PolygonSequences._polygonator(self.offset_to_polysides[s], self.circumradius)
                return self._offset_to_polysides[s]
        else:
#             print(type(s))
            start, stop, step = s.indices(self.__len__())
#             return [PolygonSequences._polygonator(self.offset_to_polysides[i], self.circumradius) for i in range(start, stop, step)]
            return [self._offset_to_polysides[i] for i in range(start, stop, step)]
            
    def __iter__(self):
        '''
        Iterator function that returns a new iterator on every call
        '''
#         print("Calling Polygon Sequence __iter__")
        return self.PolygonSqIterator(self) 
    
    @staticmethod 
    @lru_cache(2**10) 
    def _polygonator(n_sides, circumradius):                
        if n_sides < 3:
            raise ValueError(f'Minimum number of sides for a polygon is 3')
        
        return ConvexPolygon(n_sides, circumradius)
    
    class PolygonSqIterator:
        '''
        The iterator class over Polygon sequence that converts it into an iterable.
        Implements __next__ and __iter__ functions.
        '''
        def __init__(self, polygon_sq_obj):
#             print("Calling PolygonSqIterator __init__")
            self.polygon_sq_obj = polygon_sq_obj
            self._index = 0
            
        def __iter__(self):
            '''
            Basic __iter__ method that returns an instance of the iterator
            '''
#             print("Calling PolygonSqIterator instance __iter__")
            return self
        
        
        def __next__(self):
            '''
            Implementation of __next__ function. Throws StopIteration when length is bypassed.
            '''
#             print("Calling PolygonSqIterator __next__")
            if self._index >= len(self.polygon_sq_obj):
                raise StopIteration
            else:
                item = self.polygon_sq_obj._offset_to_polysides[self._index]
                self._index += 1
                return item


In [6]:
polygon_gen = PolygonSequences(10,3)
polygon_gen

Instance of PolygonSequences class. Max Edges:10 Radius:3 Len: 8

In [7]:
print(polygon_gen)

Instance of PolygonSequences class. Max Edges:10 Radius:3 Len: 8


In [8]:
### Iterables and Iterators must have __iter__ method
### Iterables shouldn't have a next method but Iterators must have a next method.
print('__iter__' in dir(PolygonSequences))
print('__iter__' in dir(PolygonSequences.PolygonSqIterator))
print('__next__' in dir(PolygonSequences))
print('__next__' in dir(PolygonSequences.PolygonSqIterator))

True
True
False
True


In [9]:
### Check enumerate, list comprehension on iterable
print("List Comprehension:\n", [poly for poly in polygon_gen])

   
print("enumerate on iterable:\n", list(enumerate(polygon_gen)))



List Comprehension:
 [Instance of ConvexPolygon class. Edges:3 Radius:3, Instance of ConvexPolygon class. Edges:4 Radius:3, Instance of ConvexPolygon class. Edges:5 Radius:3, Instance of ConvexPolygon class. Edges:6 Radius:3, Instance of ConvexPolygon class. Edges:7 Radius:3, Instance of ConvexPolygon class. Edges:8 Radius:3, Instance of ConvexPolygon class. Edges:9 Radius:3, Instance of ConvexPolygon class. Edges:10 Radius:3]
enumerate on iterable:
 [(0, Instance of ConvexPolygon class. Edges:3 Radius:3), (1, Instance of ConvexPolygon class. Edges:4 Radius:3), (2, Instance of ConvexPolygon class. Edges:5 Radius:3), (3, Instance of ConvexPolygon class. Edges:6 Radius:3), (4, Instance of ConvexPolygon class. Edges:7 Radius:3), (5, Instance of ConvexPolygon class. Edges:8 Radius:3), (6, Instance of ConvexPolygon class. Edges:9 Radius:3), (7, Instance of ConvexPolygon class. Edges:10 Radius:3)]


In [10]:
### For loops and enume
print("For loop on iterable")
try:
    for poly in polygon_gen:
        print("Valid element:", poly)
except StopIteration:
    print("Seems like list exhausted")

For loop on iterable
Valid element: ConvexPolygon Object: Edges:3 Radius:3
Valid element: ConvexPolygon Object: Edges:4 Radius:3
Valid element: ConvexPolygon Object: Edges:5 Radius:3
Valid element: ConvexPolygon Object: Edges:6 Radius:3
Valid element: ConvexPolygon Object: Edges:7 Radius:3
Valid element: ConvexPolygon Object: Edges:8 Radius:3
Valid element: ConvexPolygon Object: Edges:9 Radius:3
Valid element: ConvexPolygon Object: Edges:10 Radius:3


In [11]:
poly_Iter = iter(polygon_gen)
while True:
    try:
        print(next(poly_Iter))
    except StopIteration:
        print("List finally exhausted")
        break

ConvexPolygon Object: Edges:3 Radius:3
ConvexPolygon Object: Edges:4 Radius:3
ConvexPolygon Object: Edges:5 Radius:3
ConvexPolygon Object: Edges:6 Radius:3
ConvexPolygon Object: Edges:7 Radius:3
ConvexPolygon Object: Edges:8 Radius:3
ConvexPolygon Object: Edges:9 Radius:3
ConvexPolygon Object: Edges:10 Radius:3
List finally exhausted


### Sequence behavior regression

In [13]:
### Checking indexing and slicing
def print_details_poly(poly_obj):
    print(str(poly_obj))
    print(f"Count of Edges: {poly_obj.get_edges} Vertices: {poly_obj.get_vertices}")
    print(f"Interior Angle: {poly_obj.get_interior_angle} Apothem: {poly_obj.get_apothem}")
    print(f"Circumradius: {poly_obj.get_circumradius} Length of Side: {poly_obj.get_length_of_side}")
    print(f"Perimenter: {poly_obj.get_perimeter} Area: {poly_obj.get_area}")

poly_one = polygon_gen[0]
print_details_poly(poly_one)
# print(poly_one)

ConvexPolygon Object: Edges:3 Radius:3
Count of Edges: 3 Vertices: 3
Interior Angle: 60.0 Apothem: 1.5000000000000004
Circumradius: 3 Length of Side: 5.196152422706632
Perimenter: 15.588457268119896 Area: 11.691342951089926


In [14]:
poly_last = polygon_gen[-1]
print_details_poly(poly_one)

ConvexPolygon Object: Edges:3 Radius:3
Count of Edges: 3 Vertices: 3
Interior Angle: 60.0 Apothem: 1.5000000000000004
Circumradius: 3 Length of Side: 5.196152422706632
Perimenter: 15.588457268119896 Area: 11.691342951089926


In [15]:
## Returns an array starting from 1st element till last but one
polygon_gen[1:-1:]

[Instance of ConvexPolygon class. Edges:4 Radius:3,
 Instance of ConvexPolygon class. Edges:5 Radius:3,
 Instance of ConvexPolygon class. Edges:6 Radius:3,
 Instance of ConvexPolygon class. Edges:7 Radius:3,
 Instance of ConvexPolygon class. Edges:8 Radius:3,
 Instance of ConvexPolygon class. Edges:9 Radius:3]

In [16]:
## Negative indexing and slicing
polygon_gen[-11:-1:]

[Instance of ConvexPolygon class. Edges:3 Radius:3,
 Instance of ConvexPolygon class. Edges:4 Radius:3,
 Instance of ConvexPolygon class. Edges:5 Radius:3,
 Instance of ConvexPolygon class. Edges:6 Radius:3,
 Instance of ConvexPolygon class. Edges:7 Radius:3,
 Instance of ConvexPolygon class. Edges:8 Radius:3,
 Instance of ConvexPolygon class. Edges:9 Radius:3]

In [17]:
### Reversed sequence
polygon_gen[::-1]

[Instance of ConvexPolygon class. Edges:10 Radius:3,
 Instance of ConvexPolygon class. Edges:9 Radius:3,
 Instance of ConvexPolygon class. Edges:8 Radius:3,
 Instance of ConvexPolygon class. Edges:7 Radius:3,
 Instance of ConvexPolygon class. Edges:6 Radius:3,
 Instance of ConvexPolygon class. Edges:5 Radius:3,
 Instance of ConvexPolygon class. Edges:4 Radius:3,
 Instance of ConvexPolygon class. Edges:3 Radius:3]